In [233]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [296]:
from glob import glob
import os
import re
import time
from cStringIO import StringIO

import numpy as np
import scipy.linalg
import scipy.stats

import emcee

import kepler
import threebody

In [235]:
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 120

In [236]:
jobid = "12710"
job_dir = glob(os.path.join("emcee-chains",jobid+".*"))[0]
d = np.load(os.path.join(job_dir,"fitted_parameters.npz"))
parameter_means = d['parameter_means'][None][0]
parameter_errors = d['parameter_errors'][None][0]
fitter_params = d['fitter_params'][None][0]
parameter_formatted = dict((p,threebody.fmt(parameter_means[p],parameter_errors[p])) for p in parameter_means)

In [267]:
delta_systematic = 7e-7

In [237]:
F = threebody.Fitter(**fitter_params)

In [238]:
mean_parameters = F.best_parameters.copy()
for p in F.best_parameters:
    if p in parameter_means:
        mean_parameters[p] = parameter_means[p]
for p in F.best_parameters:
    print p, F.best_parameters[p], mean_parameters[p]

eps1_o 0.035185962344 0.0351859482877
eps1_i 0.000688276287297 0.000688328811046
asini_i 1.21752526088 1.21752519372
eps2_i -9.1397871791e-05 -9.14005477751e-05
delta_lan 2.09678232569e-06 2.17322357668e-06
eps2_o -0.00346308155564 -0.00346312843417
q_i 0.137405319602 0.137404651646
asini_o 74.6726322588 74.6726285732
pb_o 327.256877496 327.256845337
delta -1.09917345099e-06 -1.15459020471e-06
pb_i 1.62939337126 1.62939319538
tasc_i 0.407716637597 0.407716621071
dgamma -2.02679229116e-06 5.4434103637e-08
acosi_o 91.3653052105 91.3575654591
dbeta 0.00153195348462 0.000329797389546
acosi_i 1.48962580333 1.48949680473
tasc_o 313.935151842 313.935119741


In [239]:
mean_parameters = F.best_parameters.copy()
for p in F.best_parameters:
    if p in parameter_means:
        mean_parameters[p] = parameter_means[p]
for p in F.best_parameters:
    print p, F.best_parameters[p], mean_parameters[p]

eps1_o 0.035185962344 0.0351859482877
eps1_i 0.000688276287297 0.000688328811046
asini_i 1.21752526088 1.21752519372
eps2_i -9.1397871791e-05 -9.14005477751e-05
delta_lan 2.09678232569e-06 2.17322357668e-06
eps2_o -0.00346308155564 -0.00346312843417
q_i 0.137405319602 0.137404651646
asini_o 74.6726322588 74.6726285732
pb_o 327.256877496 327.256845337
delta -1.09917345099e-06 -1.15459020471e-06
pb_i 1.62939337126 1.62939319538
tasc_i 0.407716637597 0.407716621071
dgamma -2.02679229116e-06 5.4434103637e-08
acosi_o 91.3653052105 91.3575654591
dbeta 0.00153195348462 0.000329797389546
acosi_i 1.48962580333 1.48949680473
tasc_o 313.935151842 313.935119741


In [240]:
resids = F.residuals(mean_parameters)

In [261]:
rms_resid_us = np.sqrt(np.mean(resids**2))/parameter_means['f0'] * 1e6
wts = F.phase_uncerts**(-1)
rms_resid_us_wt = np.sqrt(np.sum(wts*resids**2)/np.sum(wts))/parameter_means['f0'] * 1e6
print rms_resid_us, rms_resid_us_wt

1.54261845818 1.18183020775


In [242]:
2./(360*60*60*1000)*24*60*60

0.0001333333333333333

In [243]:
def fmt_tex(x, u):
    x = float(x)
    u = float(u)
    if u<0:
        raise ValueError("Uncertainty %g < 0" % u)
    elif u==0:
        return "%g" % x
    exponent_number = np.floor(np.log10(np.abs(x)))
    exponent_error = np.floor(np.log10(u))
    if u*10**(-exponent_error)>=2:
        ndigs = 1
    else:
        ndigs = 2
    if -1<=exponent_number<=2:
        no_exponent = True
    else:
        no_exponent = False
    if exponent_error>exponent_number:
        fstr = "$0({:"+str(ndigs)+r"d})\times 10^{{ {:d} }}$"
        return fstr.format(
            int(np.rint(u*10**(-exponent_error+(ndigs-1)))),
            int(exponent_error-ndigs+1))
    
    number_digits = int(exponent_number-exponent_error)+ndigs-1
    if no_exponent:
        fstr = "${:."+str(number_digits-int(exponent_number))+r"f}({:d})$"
        return fstr.format(
           x,
           int(np.rint(u*10**(-exponent_error+(ndigs-1)))))
    else:
        fstr = "${:."+str(number_digits)+r"f}({:d})\times 10^{{ {:d} }}$"
        return fstr.format(
           x*10**(-exponent_number),
           int(np.rint(u*10**(-exponent_error+(ndigs-1)))),
           int(exponent_number))

def fmt_mjd(x, u, base_mjd):
    x = float(x)
    u = float(u)
    if u<0:
        raise ValueError("Uncertainty %g < 0" % u)
    elif u==0:
        raise ValueError("Uncertainty %g == 0" % u)
        #return "%g" % x
    fracpart = (x-np.floor(x)) + (base_mjd-np.floor(base_mjd))
    intpart = np.floor(x)+np.floor(base_mjd)+np.floor(fracpart)
    fracpart -= np.floor(fracpart)
    exponent_error = np.floor(np.log10(u))
    if u*10**(-exponent_error)>=2:
        ndigs = 1
    else:
        ndigs = 2
    number_digits = int(-exponent_error)+ndigs-1
    fstr = "${:5d}.{:d}({:d})$"
    return fstr.format(int(intpart),
       int(np.rint(fracpart*10**number_digits)),
       int(np.rint(u*10**(-exponent_error+(ndigs-1)))))

print fmt_tex(1000,0.013)
print fmt_mjd(531.27,0.03,55920)

$1.000000(13)\times 10^{ 3 }$
$56451.27(3)$


In [244]:
for k in sorted(parameter_means.keys()):
    print k, threebody.fmt(parameter_means[k], parameter_errors[k])

B 2.2183293(13)e8
Edot 3.404490(4)e34
P 2.73258863256(3)e-3
Pdot 1.758643(2)e-20
a_i 1.92379(14)e0
a_o 1.17992(9)e2
a_tot_i 1.59247(12)e1
a_tot_o 5.8845(4)e2
acosi_i 1.48950(19)e0
acosi_o 9.1358(12)e1
asini_i 1.2175252(2)e0
asini_o 7.4672629(13)e1
d_DECJ -2.28(10)e1
d_PMDEC 2.2(2)e0
d_PMRA 4.51(6)e0
d_PX -7.5(12)e-2
d_RAJ 1.04(2)e1
dbeta 0(3)e-3
delta -1.15(20)e-6
delta_i 2.4(11)e-5
delta_i_deg 1.4(6)e-3
delta_lan 2.2(8)e-6
delta_lan_deg 1.2(4)e-4
delta_omega -3.391(3)e-2
delta_omega_deg -1.9427(16)e0
dgamma 0(2)e-5
e_i 6.9437(20)e-4
e_o 3.535596(3)e-2
eps1_i 6.8833(20)e-4
eps1_o 3.518595(5)e-2
eps2_i -9.1401(17)e-5
eps2_o -3.46313(17)e-3
f0 3.65953363080(4)e2
f1 -2.355208(3)e-15
i 6.8524(6)e-1
i_deg 3.9262(4)e1
i_i 6.8527(6)e-1
i_i_deg 3.9263(4)e1
i_o 6.8524(7)e-1
i_o_deg 3.9261(4)e1
m_1 1.9730(4)e-1
m_2 4.0962(9)e-1
m_2_over_m_p 2.85265(13)e-1
m_psr 1.4359(3)e0
m_tot 2.0428(5)e0
nordtvedt_from_delta 1.15(20)e-5
nordtvedt_parameter 0(12)e-3
omega_i 1.70281(2)e0
omega_i_deg 9.75638(14)

In [245]:
class TableEntry:
    def __init__(self, description, mathname, value, error, units, base_mjd=None):
        self.description = description
        self.mathname = mathname
        self.value = value
        self.error = error
        self.units = units
        self.base_mjd = base_mjd
        
    def as_tex(self):
        if self.base_mjd is None:
            return (self.description + " & " +
                    self.mathname + " & " +
                    fmt_tex(self.value, self.error) + " " + self.units + r"""\\
""")
        else:
            return (self.description + " & " +
                    self.mathname + " & MJD " +
                    fmt_mjd(self.value, self.error, self.base_mjd) + r"""\\
""")

In [246]:
class TableEntryString:
    def __init__(self, description, mathname, value):
        self.description = description
        self.mathname = mathname
        self.value = value
        
    def as_tex(self):
        return (self.description + " & " +
                self.mathname + " & " +
                self.value + r"""\\
""")

In [247]:
s = r"""\hline
\multicolumn{3}{c}{%s}\\
\hline
"""

In [248]:
s2 = r"""\multicolumn{3}{c}{%s}\\""" + "\n"


In [249]:
class Divider:
    def __init__(self, kind, text=None):
        self.kind = kind
        self.text = text
    def as_tex(self):
        if self.kind=="h":
            return s % self.text
        elif self.kind==" ":
            return s2 % self.text
        else:
            raise ValueError

In [361]:
fnlabels = [r"\ast",
            r"\dagger",
            r"\ddagger",
            r"\S",
            r"\parallel",
            r"\P",
            r"\#",
            #r"\star",
           ]
def format_table(entries, caption=None, kind="longtable"):
    o = StringIO()
    if kind=="longtable":
        o.write(r"""\begin{longtable}{lcc}
Parameter & Symbol & Value \\
\hline \endhead
""")
    elif kind=="minipage":
        o.write(r"""\begin{table}
""")
        if caption is not None:
            o.write(r"""\caption{%s}
""" % caption)
        o.write(r"""\begin{minipage}{\textwidth}
\begin{center}
\footnotesize
\begin{tabular}{lcc}
Parameter & Symbol & Value \\
\hline
""")
    elif kind=="submission":
        o.write(r"""\documentclass{nature}
        \usepackage{aas_macros}
        \usepackage{amsmath}
        \usepackage{amssymb}
        \usepackage{gensymb}
        \newcommand{\Sun}{\odot}

        
        \begin{document}
        \begin{center}
        \footnotesize
        \begin{tabular}{lcc}
Parameter & Symbol & Value \\
\hline
""")
    else:
        raise ValueError("Unrecognized kind '%s'" % kind)
        
    for e in entries:
        o.write(e.as_tex())
    o.write(r"""\hline
""")
    if kind=="longtable":
        if caption is not None:
            o.write(r"""\caption{%s}""" % caption)
        o.write(r"""
\end{longtable}
""")
        return o.getvalue()
    elif kind=="minipage":
        o.write(r"""\end{tabular}
\end{center}
\end{minipage}
\end{table}
""")
        return o.getvalue()
    elif kind=="submission":
        o.write(r"""
\end{tabular}
\end{center}
\end{document}
""")
        ov = o.getvalue()
        o2 = StringIO()
        o2.write(r"""
\begin{table}
""")
        if caption is not None:
            o2.write(r"""\caption{%s}

""" % caption)
        i = [0]
        def footnote(m):
            r =  "$^{%s}$" % (fnlabels[i[0]%len(fnlabels)]*(i[0]//len(fnlabels) + 1))
            i[0] += 1
            o2.write("%s%s\n\n" % (r,m.groups()[0]))
            return r
        fn_re = r"\\footnote{(.*?({.*?}.*?)*)}"
        #while True:
        #    for m in re.finditer(fn_re, ov):
        #        print m.groups()
        #    i += 1
        #    break
        ov = re.sub(fn_re, footnote, ov) 
        o2.write(r"""
\end{table}
""")
        return ov, o2.getvalue()


In [251]:

table = format_table([
    Divider("h","Fixed values"),
    TableEntryString("Right ascension",
        r"RA",r"$03^h 37^m 43^s.82589(13)$"),
    TableEntryString("Declination",
        r"Dec",r"$+17\degree 15' 14'' .828(2)$"),
    TableEntryString("Dispersion measure",
        r"DM",r"$21.3162(3)$\,pc\,cm$^{-3}$"),
    TableEntryString("Solar system ephemeris",
        r"","DE435"),
    TableEntryString("Reference epoch",
        r"","MJD %d.0" % F.base_mjd),
    TableEntryString("Observation span",
        r"","MJD %.1f--%.1f" % (F.mjds[0]+F.base_mjd,F.mjds[-1]+F.base_mjd)),
    TableEntryString("Number of TOAs",
        r"","%d" % len(F.mjds)),
    TableEntryString("Root-mean-squared residual (unweighted)",
        r"",r"$%.1f\,\mu$s" % rms_resid_us),
    Divider("h","Fitted parameters"),
    Divider(" ","Spin-down parameters"),
    #TableEntry("Zero of pulsar phase",
    #    r"$t_0$",parameter_means["tzrmjd"],parameter_errors["tzrmjd"],"MJD",base_mjd=F.base_mjd),
    TableEntry("Pulsar spin frequency",
        r"$f$",parameter_means["f0"],parameter_errors["f0"],"Hz"),
    TableEntry("Spin frequency derivative",
        r"$\dot f$",parameter_means["f1"],parameter_errors["f1"],"Hz s$^{-1}$"),
    Divider(" ","Inner Keplerian parameters for pulsar orbit"),
    TableEntry("Semimajor axis projected along line of sight",
        r"$(a \sin i)_I$",parameter_means["asini_i"],parameter_errors["asini_i"],"lt-s"),
    TableEntry("Orbital period",
        r"$P_{b,I}$",parameter_means["pb_i"],parameter_errors["pb_i"],"d"),
    TableEntry("Eccentricity parameter ($e\sin \Omega$)",
        r"$\epsilon_{1,I}$",parameter_means["eps1_i"],parameter_errors["eps1_i"],""),
    TableEntry("Eccentricity parameter ($e\cos \Omega$)",
        r"$\epsilon_{2,I}$",parameter_means["eps2_i"],parameter_errors["eps2_i"],""),
    TableEntry("Time of ascending node",
        r"$t_{\text{asc},I}$",parameter_means["tasc_i"],parameter_errors["tasc_i"],"MJD",base_mjd=F.base_mjd),
    Divider(" ","Outer Keplerian parameters for centre of mass of inner binary"),
    TableEntry("Semimajor axis projected along line of sight",
        r"$(a \sin i)_O$",parameter_means["asini_o"],parameter_errors["asini_o"],"lt-s"),
    TableEntry("Orbital period",
        r"$P_{b,O}$",parameter_means["pb_o"],parameter_errors["pb_o"],"d"),
    TableEntry("Eccentricity parameter ($e\sin \Omega$)",
        r"$\epsilon_{1,O}$",parameter_means["eps1_o"],parameter_errors["eps1_o"],""),
    TableEntry("Eccentricity parameter ($e\cos \Omega$)",
        r"$\epsilon_{2,O}$",parameter_means["eps2_o"],parameter_errors["eps2_o"],""),
    TableEntry("Time of ascending node",
        r"$t_{\text{asc},O}$",parameter_means["tasc_o"],parameter_errors["tasc_o"],"MJD",base_mjd=F.base_mjd),
    Divider(" ","Interaction parameters"),
    TableEntry("Semimajor axis projected in plane of sky",r"$(a \cos i)_I$",parameter_means["acosi_i"],parameter_errors["acosi_i"],"lt-s"),
    TableEntry("Semimajor axis projected in plane of sky",r"$(a \cos i)_O$",parameter_means["acosi_o"],parameter_errors["acosi_o"],"lt-s"),
    TableEntry("Inner companion mass over PSR mass",
        r"$q_I = m_{cI}/m_p$",parameter_means["q_i"],parameter_errors["q_i"],""),
    TableEntry("Difference in longs. of asc. nodes",
        r"$\delta_{\text{lan}}$",parameter_means["delta_lan_deg"],parameter_errors["delta_lan_deg"],"$\degree$"),
    Divider(" ","GR violation parameters"),
    TableEntry(r"PPN nonlinearity-of-gravity parameter",
        r"$\beta_{\text{PPN}}-1$",parameter_means["dbeta"],parameter_errors["dbeta"],""),
    TableEntry(r"PPN space curvature parameter",
        r"$\gamma_{\text{PPN}}-1$",parameter_means["dgamma"],parameter_errors["dgamma"],""),
    # FIXME: handle systematic errors better
    #TableEntry(r"SEP violation parameter (see text)",
    #    r"$\Delta$",parameter_means["delta"],parameter_errors["delta"],""),
    TableEntryString(r"SEP violation parameter",
        r"$\Delta$",
        r"$(%.1f \pm %.1f \pm %.1f) \times 10^{-6}$"
                     % (1e6*parameter_means["delta"],
                        1e6*parameter_errors["delta"],
                        1e6*delta_systematic)),
    Divider("h","Inferred values"),
    Divider(" ","Pulsar properties"),
    TableEntry("Pulsar period",
        r"$P$",parameter_means["P"]*1e3,parameter_errors["P"]*1e3,"ms"),
    TableEntry("Pulsar period derivative",
        r"$\dot P$",parameter_means["Pdot"],parameter_errors["Pdot"],""),
    TableEntry("Inferred surface dipole magnetic field",
        r"$B$",parameter_means["B"],parameter_errors["B"],"G"),
    TableEntry("Spin-down power",
        r"$\dot E$",parameter_means["Edot"],parameter_errors["Edot"],"erg s$^{-1}$"),
    TableEntry("Characteristic age",
        r"$\tau$",parameter_means["tau"],parameter_errors["tau"],"y"),
    TableEntryString("Inferred surface dipole magnetic field",
        r"$B$",r"$1.6\times 10^8$ G"),
    TableEntryString("Spin-down power",
        r"$\dot E$",r"$1.7\times 10^{34}$ erg s$^{-1}$"),
    TableEntryString("Characteristic age",
        r"$\tau$",r"$4.9\times 10^9$ y"),
    Divider(" ","Orbital geometry"),
    TableEntry("Pulsar semimajor axis (inner)",
        r"$a_I$",parameter_means["a_i"],parameter_errors["a_i"],"lt-s"),
    TableEntry("Eccentricity (inner)",
        r"$e_I$",parameter_means["e_i"],parameter_errors["e_i"],""),
    TableEntry("Longitude of periastron (inner)",
        r"$\omega_I$",parameter_means["omega_i_deg"],parameter_errors["omega_i_deg"],"$\degree$"),
    TableEntry("Pulsar semimajor axis (outer)",
        r"$a_O$",parameter_means["a_o"],parameter_errors["a_o"],"lt-s"),
    TableEntry("Eccentricity (outer)",
        r"$e_O$",parameter_means["e_o"],parameter_errors["e_o"],""),
    TableEntry("Longitude of periastron (outer)",
        r"$\omega_O$",parameter_means["omega_o_deg"],parameter_errors["omega_o_deg"],"$\degree$"),
    TableEntry("Inclination of invariant plane",
        r"$i$",parameter_means["i_deg"],parameter_errors["i_deg"],"$\degree$"),
    TableEntry("Inclination of inner orbit",
        r"$i_I$",parameter_means["i_i_deg"],parameter_errors["i_i_deg"],"$\degree$"),
    TableEntry("Angle between orbital planes",
        r"$\delta_i$",parameter_means["delta_i_deg"],parameter_errors["delta_i_deg"],"$\degree$"),
    TableEntry("Angle between eccentricity vectors",
        r"$\delta_\omega \sim \omega_O-\omega_I$",parameter_means["delta_omega_deg"],parameter_errors["delta_omega_deg"],"$\degree$"),
    Divider(" ","Masses"),
    TableEntry("Pulsar mass",
        r"$m_p$",parameter_means["m_psr"],parameter_errors["m_psr"],"$M_\Sun$"),
    TableEntry("Inner companion mass",
        r"$m_{cI}$",parameter_means["m_1"],parameter_errors["m_1"],"$M_\Sun$"),
    TableEntry("Outer companion mass",
        r"$m_{cO}$",parameter_means["m_2"],parameter_errors["m_2"],"$M_\Sun$"),
    ])
print table

\begin{longtable}{lcc}
Parameter & Symbol & Value \\
\hline \endhead
\hline
\multicolumn{3}{c}{Fixed values}\\
\hline
Right ascension & RA & $03^h 37^m 43^s.82589(13)$\\
Declination & Dec & $+17\degree 15' 14'' .828(2)$\\
Dispersion measure & DM & $21.3162(3)$\,pc\,cm$^{-3}$\\
Solar system ephemeris &  & DE435\\
Reference epoch &  & MJD 55920.0\\
Observation span &  & MJD 55956.7--57866.9\\
Number of TOAs &  & 27110\\
Root-mean-squared residual (unweighted) &  & $1.5\,\mu$s\\
\hline
\multicolumn{3}{c}{Fitted parameters}\\
\hline
\multicolumn{3}{c}{Spin-down parameters}\\
Pulsar spin frequency & $f$ & $365.953363080(4)$ Hz\\
Spin frequency derivative & $\dot f$ & $-2.355208(3)\times 10^{ -15 }$ Hz s$^{-1}$\\
\multicolumn{3}{c}{Inner Keplerian parameters for pulsar orbit}\\
Semimajor axis projected along line of sight & $(a \sin i)_I$ & $1.2175252(2)$ lt-s\\
Orbital period & $P_{b,I}$ & $1.6293932(6)$ d\\
Eccentricity parameter ($e\sin \Omega$) & $\epsilon_{1,I}$ & $6.8833(20)\times 10^{

In [252]:
with open("table.tex", "wt") as f:
    f.write(r"""%\documentclass{article}
%\usepackage{amsmath}
%\usepackage{gensymb}
%\newcommand{\Sun}{\odot}
%\begin{document}
% DO NOT EDIT - automatically generated
{\scriptsize
""")
    f.write(table)
    f.write(r"""
}

The system parameters for PSR~J0337+17. Values in parentheses represent 
$1\sigma$ errors in the last decimal place(s), as determined by our MCMC 
fitting. In the top section are parameter 
values we supplied as input to our timing fit; the position was determined 
with VLBI, and the DM we determined by examining a single observation. 
In the middle section are
the parameters we used to describe the state of the system as of the
reference epoch, which is when we began our differential equation 
integrations. Along with the pulsar spin-down parameters, these parameters 
are further divided into the conventional Keplerian parameters measurable 
in binary pulsars, one set for each of the orbits, plus four parameters 
that are measurable only because of orbital interactions. These fourteen 
parameters plus the longitude of the ascending node of the inner system 
and the center of mass position and velocity, all assumed to be zero, can 
completely describe any combination of three positions, velocities, and 
masses. 
The last subsection contains parameters that describe the degree to which 
this timing solution differs from GR. The PPN parameters $\beta$ and $\gamma$
are not substantially constrained by our observations, and their values and 
uncertainties are consistent with the priors we obtained from Solar System
experiments. The SEP vioaltion parameter $\Delta$ is the fractional difference
in acceleration between the pulsar and the inner white dwarf, and because it is
the focus of this paper we take additional steps to estimate the impact of 
systematics on it.
The fourth section contains quantities we inferred from the result
of our fitting. Although some of the fitting parameters are highly covariant,
we computed inferred parameters and their errors based on a sample from the
posterior distribution, correctly taking into account these covariances. 
Note that since we do not correct for the mean slowing, the pulsar spin 
frequency and its derivative are changed by a fractional amount on the 
order of $10^{-6}$, and therefore so are all quantities derived from them.
We use the standard formulae for computing $B$, $\dot E$, and $\tau$; in 
particular we assume a pulsar mass of $1.4M_\Sun$ and a moment of inertia
of $10^{45}\;g\;cm^2$.
The Laplace-Lagrange parameters $\epsilon_1$ and $\epsilon_2$ provide a 
parameterization of the eccentricity ($e$) and longitude of periastron ($\Omega$) 
of an orbit that avoids a coordinate singularity at zero eccentricity; 
the pair $(\epsilon_2,\epsilon_1)$ forms a vector in the plane of the orbit 
called the eccentricity vector.
For a single orbit, the ascending node is the place where the pulsar passes 
through the plane of the sky moving away from us; the longitude of the 
ascending node specified the orientation of the orbit on the sky. This is not 
measurable with the data we have, but the difference between the longitudes of
the ascending nodes of the two orbits is measurable through orbital interactions.
The invariant plane is the plane perpendicular to the total (orbital) angular
momentum of the triple system.
Masses are as measured by a distant observer; corrections for special 
relativity are at the $10^{-8}$ fractional level and are therefore irrelevant.

%\end{document}""")


In [253]:
"""
The zero of pulsar phase is to some degree arbitrary; it is based on 
alignment with our reference profile for the Westerbork 1400~MHz 
observations. It is also measured at the solar system barycenter and at 
infinite frequency.
"""

'\nThe zero of pulsar phase is to some degree arbitrary; it is based on \nalignment with our reference profile for the Westerbork 1400~MHz \nobservations. It is also measured at the solar system barycenter and at \ninfinite frequency.\n'

In [254]:
parameter_means

{'B': 221832925.43778103494,
 'Edot': 3.4044904684162193313e+34,
 'P': 0.0027325886325610940042,
 'Pdot': 1.7586434507960213048e-20,
 'a_i': 1.9237900983725307566,
 'a_o': 117.99239926341888637,
 'a_tot_i': 15.924699644495600278,
 'a_tot_o': 588.44992015256076767,
 'acosi_i': 1.4894968047309741587,
 'acosi_o': 91.357565459093888684,
 'asini_i': 1.2175251937167079994,
 'asini_o': 74.672628573227968554,
 'd_DECJ': -22.842529479028995887,
 'd_PMDEC': 2.1602611327406574704,
 'd_PMRA': 4.5110562050134302872,
 'd_PX': -0.07479442990385770447,
 'd_RAJ': 10.406988063827958654,
 'dbeta': 0.0003297973895463089923,
 'delta': -1.1545902047096092472e-06,
 'delta_i': 2.4436592617007399e-05,
 'delta_i_deg': 0.0014001136226350712,
 'delta_lan': 2.1732235766774294651e-06,
 'delta_lan_deg': 0.0001245165388819421494,
 'delta_omega': -0.033906256159758526114,
 'delta_omega_deg': -1.9426853770436138084,
 'dgamma': 5.443410363702728656e-08,
 'e_i': 0.00069437065931644570673,
 'e_o': 0.035355964355226145559,

In [255]:
longtable = format_table([
    Divider("h","Fixed values"),
    TableEntryString(r"Right ascension\footnote{We used the same position as in the discovery paper\cite{rsa+14}.}",
        r"RA",r"$03^h 37^m 43^s.82589$"),
    TableEntryString("Declination",
        r"Dec",r"$+17\degree 15' 14'' .828$"),
    TableEntryString(r"Parallax\footnote{We used the distance estimate, based on white dwarf modelling, given in the discovery paper\cite{rsa+14}.}",
        r"$\pi$",r"$0.770$ milliarcseconds (mas)"),
    TableEntryString(r"Dispersion measure\footnote{We determined DM from a global fit that assumed no dependence of the profile on frequency.}",
        r"DM",r"$21.315933$\,pc\,cm$^{-3}$"),
    TableEntryString("Solar system ephemeris",
        r"","DE435"),
    TableEntryString("Reference epoch",
        r"","MJD %d.0" % F.base_mjd),
    TableEntryString("Observation span",
        r"","MJD %.1f--%.1f" % (F.mjds[0]+F.base_mjd,F.mjds[-1]+F.base_mjd)),
    TableEntryString("Number of TOAs",
        r"","%d" % len(F.mjds)),
    TableEntryString("Root-mean-squared residual (unweighted)",
        r"",r"$%.1f\,\mu$s" % rms_resid_us),
    
    Divider("h",r"Fitted parameters\footnote{Values in parentheses represent $1\sigma$ errors in the last decimal place(s), as determined by our MCMC fitting.}"),
    Divider(" ","Spin-down parameters"),
    #TableEntry("Zero of pulsar phase",
    #    r"$t_0$",parameter_means["tzrmjd"],parameter_errors["tzrmjd"],"MJD",base_mjd=F.base_mjd),
    TableEntry(r"Pulsar spin frequency\footnote{Note that since we do not correct for the mean slowing, the pulsar spin frequency and its derivative are changed by a fractional amount on the order of $10^{-6}$, and therefore so are all quantities derived from them.}",
        r"$f$",parameter_means["f0"],parameter_errors["f0"],"Hz"),
    TableEntry("Pulsar spin frequency derivative",
        r"$\dot f$",parameter_means["f1"],parameter_errors["f1"],"Hz s$^{-1}$"),
    Divider(" ",r"Astrometric parameters\footnote{In order to avoid having incorrect astrometry affect our SEP test, we use derivatives to fit for offsets between fixed astrometric parameters, above, and astrometry as determined from timing. As we have not carefully analyzed systematics affecting these parameters we do not recommend further use of these timing-derived astrometric parameters.}"),
    TableEntry(r"Right ascension offset",
        r"$\Delta RA$",parameter_means["d_RAJ"],parameter_errors["d_RAJ"],"mas"),
    TableEntry("Declination offset",
        r"$\Delta DEC$",parameter_means["d_DECJ"],parameter_errors["d_DECJ"],"mas"),
    TableEntry(r"Proper motion in right ascension\footnote{Proper motion in RA is expressed as the rate of change of RA; for the speed of motion across the sky it is necessary to divide by a factor of the cosine of declination}",
        r"$\mu_{RA}$",parameter_means["d_PMRA"],parameter_errors["d_PMRA"],"mas/yr"),
    TableEntry("Proper motion in declination",
        r"$\mu_{DEC}$",parameter_means["d_PMDEC"],parameter_errors["d_PMDEC"],"mas/yr"),
    TableEntry("Parallax offset",
        r"$\Delta \pi$",parameter_means["d_PX"],parameter_errors["d_PX"],"mas"),
    Divider(" ","Inner Keplerian parameters for pulsar orbit"),
    TableEntry("Semimajor axis projected along line of sight",
        r"$(a \sin i)_I$",parameter_means["asini_i"],parameter_errors["asini_i"],"lt-s"),
    TableEntry("Orbital period",
        r"$P_{b,I}$",parameter_means["pb_i"],parameter_errors["pb_i"],"d"),
    TableEntry(r"Eccentricity parameter\footnote{The Laplace-Lagrange parameters $\epsilon_1$ and $\epsilon_2$ provide a parameterization of the eccentricity ($e$) and longitude of periastron ($\Omega$) of an orbit that avoids a coordinate singularity at zero eccentricity; the pair $(\epsilon_2,\epsilon_1)$ forms a vector in the plane of the orbit called the eccentricity vector.} ($e\sin \Omega$)",
        r"$\epsilon_{1,I}$",parameter_means["eps1_i"],parameter_errors["eps1_i"],""),
    TableEntry("Eccentricity parameter ($e\cos \Omega$)",
        r"$\epsilon_{2,I}$",parameter_means["eps2_i"],parameter_errors["eps2_i"],""),
    TableEntry("Time of ascending node",
        r"$t_{\text{asc},I}$",parameter_means["tasc_i"],parameter_errors["tasc_i"],"MJD",base_mjd=F.base_mjd),
    Divider(" ","Outer Keplerian parameters for centre of mass of inner binary"),
    TableEntry("Semimajor axis projected along line of sight",
        r"$(a \sin i)_O$",parameter_means["asini_o"],parameter_errors["asini_o"],"lt-s"),
    TableEntry("Orbital period",
        r"$P_{b,O}$",parameter_means["pb_o"],parameter_errors["pb_o"],"d"),
    TableEntry("Eccentricity parameter ($e\sin \Omega$)",
        r"$\epsilon_{1,O}$",parameter_means["eps1_o"],parameter_errors["eps1_o"],""),
    TableEntry("Eccentricity parameter ($e\cos \Omega$)",
        r"$\epsilon_{2,O}$",parameter_means["eps2_o"],parameter_errors["eps2_o"],""),
    TableEntry("Time of ascending node",
        r"$t_{\text{asc},O}$",parameter_means["tasc_o"],parameter_errors["tasc_o"],"MJD",base_mjd=F.base_mjd),
    Divider(" ","Interaction parameters"),
    TableEntry("Semimajor axis projected in plane of sky",r"$(a \cos i)_I$",parameter_means["acosi_i"],parameter_errors["acosi_i"],"lt-s"),
    TableEntry("Semimajor axis projected in plane of sky",r"$(a \cos i)_O$",parameter_means["acosi_o"],parameter_errors["acosi_o"],"lt-s"),
    TableEntry("Inner companion mass over PSR mass",
        r"$q_I = m_{cI}/m_p$",parameter_means["q_i"],parameter_errors["q_i"],""),
    TableEntry(r"Difference in longs. of asc. nodes\footnote{For a single orbit, the ascending node is the place where the pulsar passes through the plane of the sky moving away from us; the longitude of the ascending node specified the orientation of the orbit on the sky. This is not measurable with the data we have, but the difference between the longitudes of the ascending nodes of the two orbits is measurable through orbital interactions.}",
        r"$\delta_{\text{lan}}$",parameter_means["delta_lan_deg"],parameter_errors["delta_lan_deg"],"$\degree$"),
    Divider(" ","GR violation parameters"),
    TableEntry(r"PPN nonlinearity-of-gravity parameter\footnote{The PPN parameters $\beta$ and $\gamma$ are not substantially constrained by our observations, and their values and uncertainties are consistent with the priors we obtained from Solar System experiments.}",
        r"$\beta_{\text{PPN}}-1$",parameter_means["dbeta"],parameter_errors["dbeta"],""),
    TableEntry(r"PPN spacetime curvature parameter",
        r"$\gamma_{\text{PPN}}-1$",parameter_means["dgamma"],parameter_errors["dgamma"],""),
    # FIXME: handle systematic errors better
    #TableEntry(r"SEP violation parameter (see text)",
    #    r"$\Delta$",parameter_means["delta"],parameter_errors["delta"],""),
    TableEntryString(r"SEP violation parameter\footnote{The SEP violation parameter $\Delta$ is the fractional difference in acceleration between the pulsar and the inner white dwarf, and because it is the focus of this paper we take additional steps to estimate the impact of systematics on it.}",
        r"$\Delta$",
        r"$(%.1f \pm %.1f \pm %.1f) \times 10^{-6}$"
                     % (1e6*parameter_means["delta"],
                        1e6*parameter_errors["delta"],
                        1e6*delta_systematic)),

    Divider("h","Inferred values"),
    Divider(" ","Pulsar properties"),
    TableEntry("Pulsar period",
        r"$P$",parameter_means["P"]*1e3,parameter_errors["P"]*1e3,"ms"),
    TableEntry("Pulsar period derivative",
        r"$\dot P$",parameter_means["Pdot"],parameter_errors["Pdot"],""),
    #TableEntry("Inferred surface dipole magnetic field",
    #    r"$B$",parameter_means["B"],parameter_errors["B"],"G"),
    #TableEntry("Spin-down power",
    #    r"$\dot E$",parameter_means["Edot"],parameter_errors["Edot"],"erg s$^{-1}$"),
    #TableEntry("Characteristic age",
    #    r"$\tau$",parameter_means["tau"],parameter_errors["tau"],"y"),
    TableEntryString(r"Inferred surface dipole magnetic field\footnote{We use the standard formulae for computing $B$, $\dot E$, and $\tau$; in particular we assume a pulsar mass of $1.4M_\Sun$ and a moment of inertia of $10^{45}\;g\;cm^2$.}",
        r"$B$",r"$2.2\times 10^8$ G"),
    TableEntryString("Spin-down power",
        r"$\dot E$",r"$3.4\times 10^{34}$ erg s$^{-1}$"),
    TableEntryString("Characteristic age",
        r"$\tau$",r"$2.5\times 10^9$ y"),
    Divider(" ","Orbital geometry"),
    TableEntry("Pulsar semimajor axis (inner)",
        r"$a_I$",parameter_means["a_i"],parameter_errors["a_i"],"lt-s"),
    TableEntry("Eccentricity (inner)",
        r"$e_I$",parameter_means["e_i"],parameter_errors["e_i"],""),
    TableEntry("Longitude of periastron (inner)",
        r"$\omega_I$",parameter_means["omega_i_deg"],parameter_errors["omega_i_deg"],"$\degree$"),
    TableEntry("Pulsar semimajor axis (outer)",
        r"$a_O$",parameter_means["a_o"],parameter_errors["a_o"],"lt-s"),
    TableEntry("Eccentricity (outer)",
        r"$e_O$",parameter_means["e_o"],parameter_errors["e_o"],""),
    TableEntry("Longitude of periastron (outer)",
        r"$\omega_O$",parameter_means["omega_o_deg"],parameter_errors["omega_o_deg"],"$\degree$"),
    TableEntry(r"Inclination of invariant plane\footnote{The invariant plane is the plane perpendicular to the total (orbital) angular momentum of the triple system.}",
        r"$i$",parameter_means["i_deg"],parameter_errors["i_deg"],"$\degree$"),
    TableEntry("Inclination of inner orbit",
        r"$i_I$",parameter_means["i_i_deg"],parameter_errors["i_i_deg"],"$\degree$"),
    TableEntry("Angle between orbital planes",
        r"$\delta_i$",parameter_means["delta_i_deg"],parameter_errors["delta_i_deg"],"$\degree$"),
    TableEntry("Angle between eccentricity vectors",
        r"$\delta_\omega \sim \omega_O-\omega_I$",parameter_means["delta_omega_deg"],parameter_errors["delta_omega_deg"],"$\degree$"),
    Divider(" ",r"Masses\footnote{Masses are as measured by a distant observer; corrections for special relativity are at the $10^{-8}$ fractional level and are therefore irrelevant.}"),
    TableEntry("Pulsar mass",
        r"$m_p$",parameter_means["m_psr"],parameter_errors["m_psr"],"$M_\Sun$"),
    TableEntry("Inner companion mass",
        r"$m_{cI}$",parameter_means["m_1"],parameter_errors["m_1"],"$M_\Sun$"),
    TableEntry("Outer companion mass",
        r"$m_{cO}$",parameter_means["m_2"],parameter_errors["m_2"],"$M_\Sun$"),
    ],
    caption=r"""\label{tab:solution}\textbf{Fit parameters.} 
""")
print longtable

with open("longtable.tex", "wt") as f:
    f.write(r"""% DO NOT EDIT - automatically generated
""")
    f.write(longtable)


\begin{longtable}{lcc}
Parameter & Symbol & Value \\
\hline \endhead
\hline
\multicolumn{3}{c}{Fixed values}\\
\hline
Right ascension\footnote{We used the same position as in the discovery paper\cite{rsa+14}.} & RA & $03^h 37^m 43^s.82589$\\
Declination & Dec & $+17\degree 15' 14'' .828$\\
Parallax\footnote{We used the distance estimate, based on white dwarf modelling, given in the discovery paper\cite{rsa+14}.} & $\pi$ & $0.770$ milliarcseconds (mas)\\
Dispersion measure\footnote{We determined DM from a global fit that assumed no dependence of the profile on frequency.} & DM & $21.315933$\,pc\,cm$^{-3}$\\
Solar system ephemeris &  & DE435\\
Reference epoch &  & MJD 55920.0\\
Observation span &  & MJD 55956.7--57866.9\\
Number of TOAs &  & 27110\\
Root-mean-squared residual (unweighted) &  & $1.5\,\mu$s\\
\hline
\multicolumn{3}{c}{Fitted parameters\footnote{Values in parentheses represent $1\sigma$ errors in the last decimal place(s), as determined by our MCMC fitting.}}\\
\hline
\mult

 In the top section are parameter 
values we supplied as input to our timing fit; the position and parallax are
taken from the discovery paper\cite{rsa+14}, and 
In the middle section are
the parameters we used to describe the state of the system as of the
reference epoch, which is when we began our differential equation 
integrations. Along with the pulsar spin-down parameters, these parameters 
are further divided into the conventional Keplerian parameters measurable 
in binary pulsars, one set for each of the orbits, plus four parameters 
that are measurable only because of orbital interactions. These fourteen 
parameters plus the longitude of the ascending node of the inner system 
and the center of mass position and velocity, all assumed to be zero, can 
completely describe any combination of three positions, velocities, and 
masses. 
The last subsection contains parameters that describe the degree to which 
this timing solution differs from GR.  

The fourth section contains quantities we inferred from the result
of our fitting. Although some of the fitting parameters are highly covariant,
we computed inferred parameters and their errors based on a sample from the
posterior distribution, correctly taking into account these covariances. 





## GR test paper version of the tables

In [368]:
entries = [
    #Divider("h","Fixed values"),
    TableEntryString(r"Right ascension"
        r"\footnote{We used the same position as in the discovery paper\cite{rsa+14}.}"
        r"\footnote{Although these parameters were held fixed, we fit for offsets from "
        r"these astrometric parameters; see Table~\ref{tab:solutionfitted}.}",
        r"RA",r"$03^h 37^m 43^s.82589$"),
    TableEntryString("Declination",
        r"Dec",r"$+17\degree 15' 14'' .828$"),
    TableEntryString(r"Parallax\footnote{We used the distance estimate, based on white dwarf modelling, given in the discovery paper\cite{rsa+14}.}",
        r"$\pi$",r"$0.770$ milliarcseconds (mas)"),
    TableEntryString(r"Dispersion measure\footnote{We determined DM from a global fit that assumed no dependence of the profile on frequency.}",
        r"DM",r"$21.315933$\,pc\,cm$^{-3}$"),
    TableEntryString("Solar system ephemeris",
        r"","DE435"),
    TableEntryString("Time conversion ephemeris",
        r"","IF99\cite{if99}"),
    TableEntryString("Time scale",
        r"",r"TCB\footnote{TCB is Barycentric Coordinate Time, a time scale that runs at a "
            r"slightly different rate (faster by $1.550505\times 10^{-8}$) than terrestrial "
            r"clocks because it has been corrected "
            r"for the gravitational time dilation due to the Solar System potential.}"),
    TableEntryString("Reference epoch",
        r"","MJD %d.0" % F.base_mjd),
    TableEntryString("Observation span",
        r"","MJD %.1f--%.1f" % (F.mjds[0]+F.base_mjd,F.mjds[-1]+F.base_mjd)),
    TableEntryString("Number of TOAs",
        r"","%d" % len(F.mjds)),
    TableEntryString("Root-mean-squared residual (weighted)",
        r"",r"$%.1f\,\mu$s" % rms_resid_us_wt),
    ]
caption = r"""\label{tab:solutionfixed}\textbf{Fixed values and characteristics of the data set.}"""
tablea = format_table(entries,
    caption=caption, 
    kind="minipage")
print tablea

with open("tablea.tex", "wt") as f:
    f.write(r"""% DO NOT EDIT - automatically generated
""")
    f.write(tablea)


\begin{table}
\caption{\label{tab:solutionfixed}\textbf{Fixed values and characteristics of the data set.}}
\begin{minipage}{\textwidth}
\begin{center}
\footnotesize
\begin{tabular}{lcc}
Parameter & Symbol & Value \\
\hline
Right ascension\footnote{We used the same position as in the discovery paper\cite{rsa+14}.}\footnote{Although these parameters were held fixed, we fit for offsets from these astrometric parameters; see Table~\ref{tab:solutionfitted}.} & RA & $03^h 37^m 43^s.82589$\\
Declination & Dec & $+17\degree 15' 14'' .828$\\
Parallax\footnote{We used the distance estimate, based on white dwarf modelling, given in the discovery paper\cite{rsa+14}.} & $\pi$ & $0.770$ milliarcseconds (mas)\\
Dispersion measure\footnote{We determined DM from a global fit that assumed no dependence of the profile on frequency.} & DM & $21.315933$\,pc\,cm$^{-3}$\\
Solar system ephemeris &  & DE435\\
Time conversion ephemeris &  & IF99\cite{if99}\\
Time scale &  & TCB\footnote{TCB is Barycentric Coor

In [369]:
o1, o2 = format_table(entries, caption=caption, kind="submission")
print o1
print
print o2
with open("tablea-standalone.tex","wt") as f1:
    with open("tablea-included.tex","wt") as f2:
        f1.write(o1)
        f2.write(o2)

\documentclass{nature}
        \usepackage{aas_macros}
        \usepackage{amsmath}
        \usepackage{amssymb}
        \usepackage{gensymb}
        \newcommand{\Sun}{\odot}

        
        \begin{document}
        \begin{center}
        \footnotesize
        \begin{tabular}{lcc}
Parameter & Symbol & Value \\
\hline
Right ascension$^{\ast}$$^{\dagger}$ & RA & $03^h 37^m 43^s.82589$\\
Declination & Dec & $+17\degree 15' 14'' .828$\\
Parallax$^{\ddagger}$ & $\pi$ & $0.770$ milliarcseconds (mas)\\
Dispersion measure$^{\S}$ & DM & $21.315933$\,pc\,cm$^{-3}$\\
Solar system ephemeris &  & DE435\\
Time conversion ephemeris &  & IF99\cite{if99}\\
Time scale &  & TCB$^{\parallel}$\\
Reference epoch &  & MJD 55920.0\\
Observation span &  & MJD 55956.7--57866.9\\
Number of TOAs &  & 27110\\
Root-mean-squared residual (weighted) &  & $1.2\,\mu$s\\
\hline

\end{tabular}
\end{center}
\end{document}



\begin{table}
\caption{\label{tab:solutionfixed}\textbf{Fixed values and characteristics of the 

In [370]:
entries = [
    #Divider("h",r"Fitted parameters\footnote{Values in parentheses represent $1\sigma$ errors in the last decimal place(s), as determined by our MCMC fitting.}"),
    Divider(" ","Pulsar spin parameters"),
    #TableEntry("Zero of pulsar phase",
    #    r"$t_0$",parameter_means["tzrmjd"],parameter_errors["tzrmjd"],"MJD",base_mjd=F.base_mjd),
    TableEntry(r"Pulsar spin frequency"
               r"\footnote{Values in parentheses represent $1\sigma$ errors in the last decimal place(s), "
               r"as determined by our MCMC fitting.}"
               r"\footnote{The pulsar's proper time scale is slowed due to both gravitational "
               r"time dilation and the transverse Doppler effect, in a way that varies as it "
               r"moves around its orbit. Counter to the usual practice in pulsar timing, we "
               r"do not correct this time scale so its average rate matches that on Earth.}",
        r"$f$",parameter_means["f0"],parameter_errors["f0"],"Hz"),
    TableEntry("Pulsar spin frequency derivative",
        r"$\dot f$",parameter_means["f1"],parameter_errors["f1"],"Hz s$^{-1}$"),
    Divider(" ",r"Astrometric parameters\footnote{In order to avoid having incorrect astrometry affect our SEP test, we use derivatives to fit for offsets between fixed astrometric parameters, above, and astrometry as determined from timing. As we have not carefully analyzed systematics affecting these parameters we do not recommend further use of these timing-derived astrometric parameters.}"),
    TableEntry(r"Right ascension offset",
        r"$\Delta RA$",parameter_means["d_RAJ"],parameter_errors["d_RAJ"],"mas"),
    TableEntry("Declination offset",
        r"$\Delta DEC$",parameter_means["d_DECJ"],parameter_errors["d_DECJ"],"mas"),
    TableEntry(r"Proper motion in right ascension",
        r"$\mu_{RA}$",parameter_means["d_PMRA"],parameter_errors["d_PMRA"],"mas/yr"),
    TableEntry("Proper motion in declination",
        r"$\mu_{DEC}$",parameter_means["d_PMDEC"],parameter_errors["d_PMDEC"],"mas/yr"),
    TableEntry("Parallax offset",
        r"$\Delta \pi$",parameter_means["d_PX"],parameter_errors["d_PX"],"mas"),
    Divider(" ","Inner Keplerian parameters for pulsar orbit"),
    TableEntry("Semi-major axis projected along line of sight",
        r"$(a \sin i)_I$",parameter_means["asini_i"],parameter_errors["asini_i"],"lt-s"),
    TableEntry("Orbital period",
        r"$P_{b,I}$",parameter_means["pb_i"],parameter_errors["pb_i"],"d"),
    TableEntry(r"Eccentricity parameter\footnote{The Laplace-Lagrange parameters\cite{lk04} $\epsilon_1$ and $\epsilon_2$ provide a parameterization of the eccentricity ($e$) and longitude of periastron ($\Omega$) of an orbit that avoids a coordinate singularity at zero eccentricity; the pair $(\epsilon_2,\epsilon_1)$ forms a vector in the plane of the orbit called the eccentricity vector.} ($e\sin \omega$)",
        r"$\epsilon_{1,I}$",parameter_means["eps1_i"],parameter_errors["eps1_i"],""),
    TableEntry("Eccentricity parameter ($e\cos \omega$)",
        r"$\epsilon_{2,I}$",parameter_means["eps2_i"],parameter_errors["eps2_i"],""),
    TableEntry("Time of ascending node",
        r"$t_{\text{asc},I}$",parameter_means["tasc_i"],parameter_errors["tasc_i"],"MJD",base_mjd=F.base_mjd),
    Divider(" ","Outer Keplerian parameters for centre of mass of inner binary"),
    TableEntry("Semi-major axis projected along line of sight",
        r"$(a \sin i)_O$",parameter_means["asini_o"],parameter_errors["asini_o"],"lt-s"),
    TableEntry("Orbital period",
        r"$P_{b,O}$",parameter_means["pb_o"],parameter_errors["pb_o"],"d"),
    TableEntry("Eccentricity parameter ($e\sin \omega$)",
        r"$\epsilon_{1,O}$",parameter_means["eps1_o"],parameter_errors["eps1_o"],""),
    TableEntry("Eccentricity parameter ($e\cos \omega$)",
        r"$\epsilon_{2,O}$",parameter_means["eps2_o"],parameter_errors["eps2_o"],""),
    TableEntry("Time of ascending node",
        r"$t_{\text{asc},O}$",parameter_means["tasc_o"],parameter_errors["tasc_o"],"MJD",base_mjd=F.base_mjd),
    Divider(" ","Orbital interaction parameters"),
    TableEntry("Semi-major axis projected in plane of sky",r"$(a \cos i)_I$",parameter_means["acosi_i"],parameter_errors["acosi_i"],"lt-s"),
    TableEntry("Semi-major axis projected in plane of sky",r"$(a \cos i)_O$",parameter_means["acosi_o"],parameter_errors["acosi_o"],"lt-s"),
    TableEntry("Ratio of inner comp. mass to PSR mass",
        r"$q_I = m_{cI}/m_p$",parameter_means["q_i"],parameter_errors["q_i"],""),
    TableEntry(r"Difference in longs. of asc. nodes\footnote{For a single orbit, the ascending node is the place where the pulsar passes through the plane of the sky moving away from us; the longitude of the ascending node specifies the orientation of the orbit on the sky. This is not measurable with the data we have, but the difference between the longitudes of the ascending nodes of the two orbits is measurable through orbital interactions.}",
        r"$\delta_{\text{lan}}$",parameter_means["delta_lan_deg"],parameter_errors["delta_lan_deg"],"$\degree$"),
    Divider(" ","GR violation parameters"),
    TableEntry(r"PPN nonlinearity-of-gravity parameter\footnote{The PPN parameters $\beta$ and $\gamma$ are not substantially constrained by our observations, and their values and uncertainties are consistent with the priors we obtained from Solar System experiments.}",
        r"$\beta_{\text{PPN}}-1$",parameter_means["dbeta"],parameter_errors["dbeta"],""),
    TableEntry(r"PPN spacetime curvature parameter",
        r"$\gamma_{\text{PPN}}-1$",parameter_means["dgamma"],parameter_errors["dgamma"],""),
    # FIXME: handle systematic errors better
    #TableEntry(r"SEP violation parameter (see text)",
    #    r"$\Delta$",parameter_means["delta"],parameter_errors["delta"],""),
    TableEntryString(r"SEP violation parameter\footnote{The SEP violation parameter $\Delta$ is the fractional difference in acceleration between the pulsar and the inner white dwarf, and because it is the focus of this paper we take additional steps to estimate the impact of systematics on it.}",
        r"$\Delta$",
        r"$(%.1f \pm %.1f) \times 10^{-6}$"
                     % (1e6*F.best_parameters["delta"],
                        1e6*delta_systematic)),
    ]
caption = r"""\label{tab:solutionfitted}\textbf{Fitted values.}"""
tableb = format_table(entries,
    caption=caption, 
    kind="minipage")
print tableb

with open("tableb.tex", "wt") as f:
    f.write(r"""% DO NOT EDIT - automatically generated
""")
    f.write(tableb)



\begin{table}
\caption{\label{tab:solutionfitted}\textbf{Fitted values.}}
\begin{minipage}{\textwidth}
\begin{center}
\footnotesize
\begin{tabular}{lcc}
Parameter & Symbol & Value \\
\hline
\multicolumn{3}{c}{Pulsar spin parameters}\\
Pulsar spin frequency\footnote{Values in parentheses represent $1\sigma$ errors in the last decimal place(s), as determined by our MCMC fitting.}\footnote{The pulsar's proper time scale is slowed due to both gravitational time dilation and the transverse Doppler effect, in a way that varies as it moves around its orbit. Counter to the usual practice in pulsar timing, we do not correct this time scale so its average rate matches that on Earth.} & $f$ & $365.953363080(4)$ Hz\\
Pulsar spin frequency derivative & $\dot f$ & $-2.355208(3)\times 10^{ -15 }$ Hz s$^{-1}$\\
\multicolumn{3}{c}{Astrometric parameters\footnote{In order to avoid having incorrect astrometry affect our SEP test, we use derivatives to fit for offsets between fixed astrometric parameters,

In [371]:
o1, o2 = format_table(entries, caption=caption, kind="submission")
print o1
print
print o2
with open("tableb-standalone.tex","wt") as f1:
    with open("tableb-included.tex","wt") as f2:
        f1.write(o1)
        f2.write(o2)

\documentclass{nature}
        \usepackage{aas_macros}
        \usepackage{amsmath}
        \usepackage{amssymb}
        \usepackage{gensymb}
        \newcommand{\Sun}{\odot}

        
        \begin{document}
        \begin{center}
        \footnotesize
        \begin{tabular}{lcc}
Parameter & Symbol & Value \\
\hline
\multicolumn{3}{c}{Pulsar spin parameters}\\
Pulsar spin frequency$^{\ast}$$^{\dagger}$ & $f$ & $365.953363080(4)$ Hz\\
Pulsar spin frequency derivative & $\dot f$ & $-2.355208(3)\times 10^{ -15 }$ Hz s$^{-1}$\\
\multicolumn{3}{c}{Astrometric parameters$^{\ddagger}$}\\
Right ascension offset & $\Delta RA$ & $10.4(2)$ mas\\
Declination offset & $\Delta DEC$ & $-22.8(10)$ mas\\
Proper motion in right ascension & $\mu_{RA}$ & $4.51(6)$ mas/yr\\
Proper motion in declination & $\mu_{DEC}$ & $2.2(2)$ mas/yr\\
Parallax offset & $\Delta \pi$ & $-7.5(12)\times 10^{ -2 }$ mas\\
\multicolumn{3}{c}{Inner Keplerian parameters for pulsar orbit}\\
Semi-major axis projected along line o

In [374]:
pdot_fixed = 1.7293209e-20
pdot_fixed_error = np.hypot(1.6065442e-23,parameter_errors["Pdot"])

entries = [
    #Divider("h","Inferred values"),
    Divider(" ","Pulsar properties"),
    TableEntry("Pulsar period",
        r"$P$",parameter_means["P"]*1e3,parameter_errors["P"]*1e3,"ms"),
    TableEntry("Pulsar period derivative",
        r"$\dot P$",parameter_means["Pdot"],parameter_errors["Pdot"],""),
    TableEntry(r"Corrected pulsar period derivative"
               r"\footnote{This period derivative includes corrections for the "
               r"Shklovskii effect\cite{shkl70} (using the unreliable astrometry computed "
               r"here) and acceleration in the Galactic potential\cite{pb17,bovy15}.}",
        r"$\dot P$",pdot_fixed,pdot_fixed_error,""),    
    #TableEntry("Inferred surface dipole magnetic field",
    #    r"$B$",parameter_means["B"],parameter_errors["B"],"G"),
    #TableEntry("Spin-down power",
    #    r"$\dot E$",parameter_means["Edot"],parameter_errors["Edot"],"erg s$^{-1}$"),
    #TableEntry("Characteristic age",
    #    r"$\tau$",parameter_means["tau"],parameter_errors["tau"],"y"),
    TableEntryString(r"Inferred surface dipole magnetic field\footnote{We use the standard formulae\cite{lk04} for computing $B$, $\dot E$, and $\tau$; in particular we assume a pulsar mass of $1.4M_\Sun$ and a moment of inertia of $10^{45}\;\text{g}\;\text{cm}^2$.}",
        r"$B$",r"$2.2\times 10^8$ G"),
    TableEntryString("Spin-down power",
        r"$\dot E$",r"$3.4\times 10^{34}$ erg s$^{-1}$"),
    TableEntryString("Characteristic age",
        r"$\tau$",r"$2.5\times 10^9$ y"),
    Divider(" ","Orbital geometry"),
    TableEntry("Pulsar semi-major axis (inner)",
        r"$a_I$",parameter_means["a_i"],parameter_errors["a_i"],"lt-s"),
    TableEntry("Eccentricity (inner)",
        r"$e_I$",parameter_means["e_i"],parameter_errors["e_i"],""),
    TableEntry("Longitude of periastron (inner)",
        r"$\omega_I$",parameter_means["omega_i_deg"],parameter_errors["omega_i_deg"],"$\degree$"),
    TableEntry("Pulsar semi-major axis (outer)",
        r"$a_O$",parameter_means["a_o"],parameter_errors["a_o"],"lt-s"),
    TableEntry("Eccentricity (outer)",
        r"$e_O$",parameter_means["e_o"],parameter_errors["e_o"],""),
    TableEntry("Longitude of periastron (outer)",
        r"$\omega_O$",parameter_means["omega_o_deg"],parameter_errors["omega_o_deg"],"$\degree$"),
    TableEntry(r"Inclination of invariant plane\footnote{The invariant plane is the plane perpendicular to the total (orbital) angular momentum of the triple system.}",
        r"$i$",parameter_means["i_deg"],parameter_errors["i_deg"],"$\degree$"),
    TableEntry("Inclination of inner orbit",
        r"$i_I$",parameter_means["i_i_deg"],parameter_errors["i_i_deg"],"$\degree$"),
    TableEntry("Angle between orbital planes",
        r"$\delta_i$",parameter_means["delta_i_deg"],parameter_errors["delta_i_deg"],"$\degree$"),
    TableEntry("Angle between eccentricity vectors",
        r"$\delta_\omega \sim \omega_O-\omega_I$",parameter_means["delta_omega_deg"],parameter_errors["delta_omega_deg"],"$\degree$"),
    TableEntry("Relativistic periastron advance (inner)",
        r"$\dot\omega_I$",np.rad2deg(parameter_means["periastron_advance_i_year"]),np.rad2deg(parameter_errors["periastron_advance_i_year"]),r"$\degree/\text{year}$"),    
    TableEntry("Relativistic periastron advance (outer)",
        r"$\dot\omega_O$",np.rad2deg(parameter_means["periastron_advance_o_year"]),np.rad2deg(parameter_errors["periastron_advance_o_year"]),r"$\degree/\text{year}$"),    
    Divider(" ",r"Masses\footnote{Masses are as measured by a distant observer; corrections for special relativity are at the $10^{-8}$ fractional level and are therefore irrelevant.}"),
    TableEntry("Pulsar mass",
        r"$m_p$",parameter_means["m_psr"],parameter_errors["m_psr"],"$M_\Sun$"),
    TableEntry("Inner companion mass",
        r"$m_{cI}$",parameter_means["m_1"],parameter_errors["m_1"],"$M_\Sun$"),
    TableEntry("Outer companion mass",
        r"$m_{cO}$",parameter_means["m_2"],parameter_errors["m_2"],"$M_\Sun$"),
    ]
caption = r"""\label{tab:solutioninferred}\textbf{Inferred values.}"""
tablec = format_table(entries,
    caption=caption, 
    kind="minipage")
print tablec

with open("tablec.tex", "wt") as f:
    f.write(r"""% DO NOT EDIT - automatically generated
""")
    f.write(tablec)


\begin{table}
\caption{\label{tab:solutioninferred}\textbf{Inferred values.}}
\begin{minipage}{\textwidth}
\begin{center}
\footnotesize
\begin{tabular}{lcc}
Parameter & Symbol & Value \\
\hline
\multicolumn{3}{c}{Pulsar properties}\\
Pulsar period & $P$ & $2.73258863256(3)$ ms\\
Pulsar period derivative & $\dot P$ & $1.758643(2)\times 10^{ -20 }$ \\
Corrected pulsar period derivative\footnote{This period derivative includes corrections for the Shklovskii effect\cite{shkl70} (using the unreliable astrometry computed here) and acceleration in the Galactic potential\cite{pb17,bovy15}.} & $\dot P$ & $1.7293(16)\times 10^{ -20 }$ \\
Inferred surface dipole magnetic field\footnote{We use the standard formulae\cite{lk04} for computing $B$, $\dot E$, and $\tau$; in particular we assume a pulsar mass of $1.4M_\Sun$ and a moment of inertia of $10^{45}\;\text{g}\;\text{cm}^2$.} & $B$ & $2.2\times 10^8$ G\\
Spin-down power & $\dot E$ & $3.4\times 10^{34}$ erg s$^{-1}$\\
Characteristic age & $\tau$

In [375]:
o1, o2 = format_table(entries, caption=caption, kind="submission")
print o1
print
print o2
with open("tablec-standalone.tex","wt") as f1:
    with open("tablec-included.tex","wt") as f2:
        f1.write(o1)
        f2.write(o2)

\documentclass{nature}
        \usepackage{aas_macros}
        \usepackage{amsmath}
        \usepackage{amssymb}
        \usepackage{gensymb}
        \newcommand{\Sun}{\odot}

        
        \begin{document}
        \begin{center}
        \footnotesize
        \begin{tabular}{lcc}
Parameter & Symbol & Value \\
\hline
\multicolumn{3}{c}{Pulsar properties}\\
Pulsar period & $P$ & $2.73258863256(3)$ ms\\
Pulsar period derivative & $\dot P$ & $1.758643(2)\times 10^{ -20 }$ \\
Corrected pulsar period derivative$^{\ast}$ & $\dot P$ & $1.7293(16)\times 10^{ -20 }$ \\
Inferred surface dipole magnetic field$^{\dagger}$ & $B$ & $2.2\times 10^8$ G\\
Spin-down power & $\dot E$ & $3.4\times 10^{34}$ erg s$^{-1}$\\
Characteristic age & $\tau$ & $2.5\times 10^9$ y\\
\multicolumn{3}{c}{Orbital geometry}\\
Pulsar semi-major axis (inner) & $a_I$ & $1.92379(14)$ lt-s\\
Eccentricity (inner) & $e_I$ & $6.9437(20)\times 10^{ -4 }$ \\
Longitude of periastron (inner) & $\omega_I$ & $97.5638(14)$ $\degree$\\

In [259]:
import astropy.units as u
import astropy.constants as c

r_p = 118*u.s*c.c
r_o = r_p * (1.4359+0.19730)/0.40962

r = r_p+r_o
r.to(u.AU)

<Quantity 1.179305288407317 AU>

In [260]:
a = c.G*(0.40962*u.solMass)/r**2

a.decompose()

<Quantity 0.0017470697535318478 m / s2>

In [271]:
pb_1713 = 67.825*u.day
pbd_1713 = 0.34e-12
a_1713_los = c.c*pbd_1713/pb_1713
a_1713_los.to(u.m/u.s**2)

<Quantity 1.7393864199806144e-11 m / s2>

In [285]:

F.pulses[-1]

55867043780.0

In [286]:
from astropy.time import Time

In [292]:
tlast = Time(57887.0,0.82729159432901155924, format="mjd")
tlast.format = "iso"
tlast.precision = 7
tlast

<Time object: scale='utc' format='iso' value=2017-05-14 19:51:17.9937500>